### get geography information (area and land area) from Census Tigerweb, pull in population and housing Decennial Census counts and calculate population and housing density in 2010 and 2020

In [24]:
import pandas as pd
import numpy as np

In [25]:
import geo_getter as get

## urban villages - blocks

In [26]:
blk_10 = pd.read_csv('../data/geo/blk_vil_10_reduced.csv')
blk_20 = pd.read_csv('../data/geo/blk_vil_20_reduced.csv')

blk_10 = blk_10[['name','land_acre','aland']].copy()
blk_10 = blk_10.groupby(['name']).sum().reset_index()
blk_10.rename(columns={'land_acre':'AREALAND_ACRE10','aland':'AREALAND10'},\
                        inplace=True)

blk_20 = blk_20[['name','land_acre','aland']].copy()
blk_20 = blk_20.groupby(['name']).sum().reset_index()
blk_20.rename(columns={'land_acre':'AREALAND_ACRE20','aland':'AREALAND20'},\
                        inplace=True)

In [27]:
blk = pd.merge(blk_10,blk_20,how='left',on='name')

In [28]:
bg_10 = pd.read_csv('../data/geo/bgp_vil_10.csv')
bg_20 = pd.read_csv('../data/geo/bgp_vil_20.csv')

In [29]:
uv10 = bg_10[['name','land_acre','aland']].copy()
uv10 = uv10.groupby(['name']).sum().reset_index()
uv10.rename(columns={'land_acre':'AREALAND_ACRE10','aland':'AREALAND10'},\
                        inplace=True)

In [30]:
uv20 = bg_20[['name','land_acre','aland']].copy()
uv20 = uv20.groupby(['name']).sum().reset_index()
uv20.rename(columns={'land_acre':'AREALAND_ACRE20','aland':'AREALAND20'},\
                        inplace=True)

In [31]:
uv = pd.merge(uv10,uv20,how='left',on='name')
uv['name'] = uv['name']+'_bg'

## cities

In [32]:
cities = {455000:'Phoenix',473000:'Tempe',465000:'Scottsdale',427820:'Glendale',\
          4865000:'San Antonio', 4819000:'Dallas',1235000:'Jacksonville',\
          1836000:'Indianapolis'}

geoids = list(cities.keys())

In [33]:
pl10 = get.make_city('2010')
pl20 = get.make_city('2020')

In [34]:
places = pd.merge(pl10,pl20,how='left',on='GEOID')

In [35]:
places = places[places.GEOID.isin(list(cities.keys()))].copy()

In [36]:
places.rename(columns={'BASENAME10':'name'},inplace=True)
places = places.drop(['GEOID','AREAWATER10','BASENAME20','AREAWATER20'],axis=1)

## maricopa

In [37]:
mar10 = get.make_maricopa('2010')
mar20 = get.make_maricopa('2020')

In [38]:
mar = pd.merge(mar10,mar20,how='left',on='GEOID')

In [39]:
mar.rename(columns={'BASENAME10':'name'},inplace=True)
mar = mar.drop(['GEOID','AREAWATER10','BASENAME20','AREAWATER20'],axis=1)

## arizona

In [40]:
az10 = get.make_az('2010')
az20 = get.make_az('2020')
az = pd.merge(az10,az20,how='left',on='GEOID')
az['name'] = 'AZ'

In [41]:
az = az.drop(['GEOID','AREAWATER10','BASENAME10','BASENAME20','AREAWATER20'],axis=1)

## us

In [42]:
us10 = get.make_us('2010')
us20 = get.make_us('2020')
us = pd.merge(us10,us20,how='left',on='FUNCSTAT')

In [43]:
us['name'] = 'US'
us = us.drop(['FUNCSTAT','AREAWATER10','AREAWATER20'],axis=1)

## make big geography table - no urban vil

In [44]:
geos = pd.concat([blk,places,mar,az,us,uv])

## read in pop and housing from output folder

In [45]:
data = pd.read_csv('output/dec_pop_hou_race.csv')

In [46]:
data = data[['name','Hou_10E','Hou_20E','Pop_10E','Pop_20E']]

## calc housing and pop density in 2010 and 2020

In [47]:
geos = pd.merge(data,geos,how='left',on='name')

In [48]:
geos['HouDen_10'] = geos.Hou_10E / geos.AREALAND_ACRE10
geos['PopDen_10'] = geos.Pop_10E / geos.AREALAND_ACRE10
geos['HouDen_20'] = geos.Hou_20E / geos.AREALAND_ACRE20
geos['PopDen_20'] = geos.Pop_20E / geos.AREALAND_ACRE20

In [49]:
geos.to_csv('output/dec_pop_hou_den.csv',index=False)